# Experimiento de reproducción modelo paper arango-resources

Esto es una reproducción y adaptación del c+odigo encontrado en el repositorio:

https://github.com/TharinduDR/DeepOffense

Asociado a la publicación:

***Multilingual Offensive Language Identification with Cross-lingual Embeddings***

In [1]:
!pip install datasets
!pip install transformers
!pip install scipy --upgrade

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
#from transformers import AutoTokenizer, AutoModelForSequenceClassification, AutoModelForMaskedLM
#from transformers import TrainingArguments, Trainer, BertTokenizer, BertForSequenceClassification
#from transformers import EarlyStoppingCallback, get_linear_schedule_with_warmup
from torch.utils.data import DataLoader
from torch.cuda.amp import GradScaler, autocast
#from datasets import load_dataset,Dataset,DatasetDict
#from transformers import DataCollatorWithPadding, AutoModelForSequenceClassification, Trainer, TrainingArguments, AutoTokenizer, AutoModel, AutoConfig, AdamW
#from transformers.modeling_outputs import TokenClassifierOutput
import torch 
import torch.nn as nn
from tqdm.notebook import tqdm
import pandas as pd
import platform
import numpy as np
#from datasets import load_dataset, Dataset, DatasetDict
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, roc_auc_score
import warnings
from sklearn.model_selection import StratifiedKFold
warnings.filterwarnings("ignore")
import seaborn as sns; sns.set()
import matplotlib.pyplot as plt
from torch import manual_seed, device, cuda, backends
import random

## Datos

In [3]:
google_drive = True
train_models = False

if google_drive:
  print("Mounting your Google Drive ...")

  from google.colab import drive
  drive.flush_and_unmount()
  drive.mount('/content/drive', force_remount=True)

Mounting your Google Drive ...
Mounted at /content/drive


In [4]:
# We set our Seeds
random_state = 42 # our beloved seed the answer to the universe
seed = np.random.seed(random_state) # set numpy seed
manual_seed(random_state) # set torch seed
random.seed(random_state) # set python seed
# set torch Cuda seeds
cuda.manual_seed(random_state)
cuda.manual_seed_all(random_state)
backends.cudnn.benchmark = False
backends.cudnn.deterministic = True

# CUDA Check
device_type = device("cuda" if cuda.is_available() else "cpu")
if cuda.is_available():
        print("[INFO] Using GPU: {}\n".format(cuda.get_device_name()))
else:
        print("[INFO] GPU not found. Using CPU: {}\n".format(platform.processor()))

[INFO] Using GPU: Tesla T4



In [5]:
test_file = 'tweets_test.csv'
train_file = 'tweets_train.csv'
referenced = 'referenced_tweets_data.csv'
# Arango-resources
train_arango = 'aarango_HS.xlsx'
# hateval2019
train_hateval_es = "hateval2019/hateval2019_es_train.csv"
test_hateval_es = "hateval2019/hateval2019_es_test.csv"
train_hateval_en = "hateval2019/hateval2019_en_train.csv"
test_hateval_en = "hateval2019/hateval2019_en_test.csv"
# Hatecheck
test_hatecheck = "hatecheck_cases_final_spanish.csv"

if google_drive:
  path = "/content/drive/MyDrive/Paper_dataton/Datos/" # Esta es la ruta
  save_path = "/content/drive/MyDrive/Paper_dataton/models_reproduction/" # Esta es la ruta de los experimentos
else:
  path = ''

data_train = pd.read_csv(path + train_file, encoding='utf-8', index_col=0)
data_test = pd.read_csv(path + test_file, encoding='utf-8', index_col=0)
data_referenced = pd.read_csv(path + referenced, encoding='utf-8', index_col=0)
data_train.rename(columns={'Odio':'HS'}, inplace = True)
data_test.rename(columns={'Odio':'HS'}, inplace = True)
# Arango
data_arango_train = pd.read_excel(path + train_arango).dropna(subset=['Usuario'])
# preprocess arango columns
data_arango_train.rename(columns={'tweet a etiquetar':'text', '12. Estereotipo':'HS'}, inplace = True)
data_arango_train
# Hateval
data_hateval_es_train = pd.read_csv(path + train_hateval_es)
data_hateval_es_test = pd.read_csv(path + test_hateval_es)
data_hateval_en_train = pd.read_csv(path + train_hateval_en)
data_hateval_en_test = pd.read_csv(path + test_hateval_en)
# Hatecheck
data_hatecheck = pd.read_csv(path + test_hatecheck)
data_hatecheck = data_hatecheck.drop(columns=['ref_case_id', 	'ref_templ_id', 	'templ_id', 	'case_templ', 	'gender_male', 	'gender_female', 	'label_annotated', 	'label_annotated_maj', 	'disagreement_in_case', 	'disagreement_in_template'])
data_hatecheck['HS'] = data_hatecheck['label_gold'].apply(lambda x: 1 if x == 'hateful' else 0)
data_hatecheck.rename(columns={'test_case':'text'}, inplace = True)


# print('Amount of different Authors: ' + 
# print(data_train.author_id.value_counts())


print('Len Arango: {}\nLen hateval es: {}\nLen hateval en: {}'.format(len(data_arango_train),
                                                                      len(data_hateval_es_train) + len(data_hateval_es_test),
                                                                      len(data_hateval_en_train) + len(data_hateval_en_test)))
display(data_train.head())
display(data_arango_train.head())
display(data_hateval_es_train.head())
display(data_hateval_en_train.head())
display(data_hatecheck.head())

Len Arango: 9834
Len hateval es: 6100
Len hateval en: 12000


,tweet_id,author_id,conversation_id,text,HS,Mujeres,Comunidad LGBTQ+,Comunidades Migrantes,Pueblos Originarios
0,1399516036240662528,1329989512391438336,(),En una amistad o soy tipo: \r\nLo peor es que ...,0,0,0,0,0
1,1320788179721560065,1319131581949378560,(),QUIEN FUE LA MARACA CULIA TE VOY A MATAR PERRA...,1,1,0,0,0
2,1079889645280944129,2328230546,(),menos mal q se recibe el año con ropa blanca j...,0,0,0,0,0
3,1369254390134145033,819218347049029633,"(1369117564811550720,)",Cuantos INFILTRADOS extranjeros hay ahi😡 https...,1,0,0,1,0
4,1533854824378290176,1229941558411243520,(),#ENCONTRADO #GUAYAQUIL\r\nFecha: 06/06/22\r\nS...,0,0,0,0,0


,CASO,Usuario,Link,text,Contexto,1. Anonimo,2. Genero,3. Mención migración,4. Mención Venezuela,5. Mención política nacional,...,10. Grosería s/Int.,11. Insulto/Sobrenombre,HS,13. Sarcasmo/Ironía/Burla,14. Pregunta Legítima,15. Evidencia,16. Figura femenina,17. Figura masculina,Codificador,Fecha
0,23036.0,Luna,https://twitter.com/labrigidagon/status/140128...,Eran tan pero tan feministas que invisibilizab...,NaN,1.0,2.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,Ariel,semana 1
1,23057.0,Karen🌼,https://twitter.com/karenseguelr/status/140270...,@Eneatipo7 @Cooperativa @karina_ol Me carga en...,NaN,1.0,2.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,Ariel,semana 1
4,23058.0,Patria Y Libertad,https://twitter.com/patrianueva2020/status/140...,", ¿Sabrán las femiorcas como @karina_ol y todo...",NaN,1.0,3.0,0.0,0.0,1.0,...,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,Ariel,semana 1
5,23059.0,Nubecita.Roja,https://twitter.com/RojaNubecita/status/140247...,@AntoDelLaw @jonaovalle @CarolinaLeitao @penal...,NaN,1.0,3.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,Ariel,semana 1
9,23061.0,Rodrigo Catalan,https://twitter.com/rodrigo140970/status/14017...,@Ciudadanolevi1 Debajo de que piedra estaba es...,NaN,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,Ariel,semana 1


,id,text,HS,TR,AG
0,20001,Easyjet quiere duplicar el número de mujeres p...,1,0,0
1,20002,El gobierno debe crear un control estricto de ...,1,0,0
2,20003,Yo veo a mujeres destruidas por acoso laboral ...,0,0,0
3,20004,"— Yo soy respetuoso con los demás, sólamente l...",0,0,0
4,20007,Antonio Caballero y como ser de mal gusto e ig...,0,0,0


,id,text,HS,TR,AG
0,201,"Hurray, saving us $$$ in so many ways @potus @...",1,0,0
1,202,Why would young fighting age men be the vast m...,1,0,0
2,203,@KamalaHarris Illegals Dump their Kids at the ...,1,0,0
3,204,NY Times: 'Nearly All White' States Pose 'an A...,0,0,0
4,205,Orban in Brussels: European leaders are ignori...,0,0,0


,mhc_case_id,functionality,text,label_gold,target_ident,HS
0,spanish-1,derog_neg_emote_h,Odio a los negros.,hateful,black people,1
1,spanish-2,derog_neg_emote_h,Odio a los gays.,hateful,gay people,1
2,spanish-3,derog_neg_emote_h,Odio a los trans.,hateful,trans people,1
3,spanish-4,derog_neg_emote_h,Odio a los indígenas.,hateful,indigenous people,1
4,spanish-5,derog_neg_emote_h,Odio a los judíos.,hateful,jews,1


In [ ]:
# Generamos los datasets enteros
data_hateval_es_full = pd.concat([data_hateval_es_train, data_hateval_es_test])
data_hateval_en_full = pd.concat([data_hateval_en_train, data_hateval_en_test])
data_full = pd.concat([data_train, data_test])

# CL1 + CL2
keep_cols = ['text', 'HS', 'text_translated']
drop_columns_ours = [col for col in data_full.columns.tolist() if col not in keep_cols]
drop_columns_arango = [col for col in data_arango_train.columns.tolist() if col not in keep_cols]

data_chilean = pd.concat([data_full.drop(columns=drop_columns_ours),
                          data_arango_train.drop(columns=drop_columns_arango)]).reset_index(drop=True)

La supuesta libreria que describen en el paper es un desastre ademas  de inexistente e irreproducible, por lo cual solo me puedo guiar del paper donde solo describen que usan **"*XLM-R classification model is trained using data
from one language, and the weights are saved.
Then, these weights are used to initialize the model
and predict labels in a different language*"**.


De esto yo solo entiendo que se entrena 'xlm-roberta' en un lenguaje sobre el que se hace fine-tuning y luego utilizan estos pesos para inicializar el mismo modelo y se predice sobre este otro lenguaje.

Sin embargo baansdome en el codigo del repositorio lo que hacen efectivamente es entrenar sobre bert-base-multilingual guardan sus pesos y luego inicializan xmlroberta sobre estos pesos

## XML-Roberta

In [ ]:
class TweetsDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels=None):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        if self.labels:
            item["labels"] = torch.tensor([self.labels[idx]])
        return item

    def __len__(self):
        return len(self.encodings["input_ids"])


# Define Evaluation metrics
def compute_metrics(p):
    pred, labels = p
    pred = np.argmax(pred, axis=1)
    accuracy = accuracy_score(y_true=labels, y_pred=pred)
    recall = recall_score(y_true=labels, y_pred=pred)
    precision = precision_score(y_true=labels, y_pred=pred)
    f1 = f1_score(y_true=labels, y_pred=pred)
    auc = roc_auc_score(y_true=labels, y_score=pred)

    return {"accuracy": accuracy, "precision": precision,
            "recall": recall, "f1": f1, "AUC": auc}

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# cl_1 Arango
# cl_2 Benoit y Ñanculef
# crear un diccionario que use como llave el lenguaje (en, es, cl) y tenga dentro train_data, eval_datasets y los path para guardar los resultados
experiments = [
    {'es': {'data_train': data_hateval_es_full,}},
    {'en': {'data_train': data_hateval_en_full,}},
    {'cl1': {'data_train': data_arango_train,}},
    {'cl2': {'data_train': data_full,}}
    ]

def train_models(experiment):
  for lang, data in tqdm(experiment.items()):
    # Loading models
    tokenizer = AutoTokenizer.from_pretrained("xlm-roberta-large", use_fast=True, return_tensors='pt')
    model = AutoModelForSequenceClassification.from_pretrained("xlm-roberta-large", num_labels=2)

    data_set = data['data_train']
    # Preprocess data
    X = list(data_set['text'])
    y = list(data_set['HS'].apply(int))

    # train and validation split
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=random_state)

    # Tokenize data
    X_train_tokenized = tokenizer(X_train, padding=True, truncation=True, max_length=128)
    X_val_tokenized = tokenizer(X_val, padding=True, truncation=True, max_length=128)

    # Create torch Dataset
    train_dataset = TweetsDataset(X_train_tokenized, y_train)
    eval_dataset = TweetsDataset(X_val_tokenized, y_val)

    model.to(device_type)
    # Define Trainer args
    args = TrainingArguments(
        output_dir=save_path + "models/" + '/EMNLP20/' + lang, # to our drive/desktop folder
        evaluation_strategy="steps",
        eval_steps=100,
        logging_steps=100,
        per_device_train_batch_size=8,
        per_device_eval_batch_size=8,
        num_train_epochs=3,
        seed=random_state,
        load_best_model_at_end=True)
    
    # Define Trainer
    trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=2)]
    ) # en el codigo del repositorio solo utilizan un dataset de entrenamiento y este no es dividido para evaluarlo por esto nosotro stampoco tenemos uno

    # Train pre-trained model (Fine Tune)
    train_result = trainer.train()



for experiment in tqdm(experiments):
  train_models(experiment)
  break

### Experiments

The experiments consists in using the full set for training and testing in another language full set

*  Training in spanish and testing in english
*  Training in spanish and testing in chilean arango
*  Training in spanish and testing in chilean ours
*  Training in english and testing in spanish
*  Training in english and testing in spanish

In [ ]:
experiments_test = [
    {'es': {'data_test': [(data_hateval_en_full, 'en'),
                          (data_arango_train, 'cl1'),
                          (data_full, 'cl2')],
      'save_file': ''}},
    {'en': {'data_test': [(data_hateval_es_full, 'es'),
                          (data_arango_train, 'cl1'),
                          (data_full, 'cl2')],
            'save_file': ''}},
    {'cl1': {'data_test': [(data_hateval_es_full, 'es'),
                           (data_hateval_en_full, 'en'),
                           (data_full, 'cl2')],
             'save_file': ''}},
    {'cl2': {'data_test': [(data_hateval_es_full, 'es'),
                           (data_hateval_en_full, 'en'),
                           (data_arango_train, 'cl1')],
             'save_file': ''}}
    ]


def test_model(experiment):
  for lang, data in tqdm(experiment.items()):
    tokenizer = AutoTokenizer.from_pretrained("xlm-roberta-base", use_fast=True)
    model = AutoModelForSequenceClassification.from_pretrained(save_path + "models/" + '/EMNLP20/' + lang + "/checkpoint-480000")
    # datasets evaluation loop
    for eval_dataset, save_path_eval in tqdm(zip(data['data_tests'], data['save_paths']), total=len(data['data_tests'])):
      data_set = eval_dataset
      # Preprocess data
      X_test = list(data_set['text'])
      y_test = list(data_set['HS'])
      # Tokenize data
      X_test_tokenized = tokenizer(X_test, padding=True, truncation=True, max_length=512)
      eval_dataset = TweetsDataset(X_test_tokenized, y_test)
      # compute evaluation results
      metrics = trainer.predict(eval_dataset)
      max_eval_samples = len(eval_dataset)
      metrics["eval_samples"] = min(max_eval_samples, len(eval_dataset))
      # save evaluation results
      trainer.log_metrics(save_path_eval, metrics)
      trainer.save_metrics(save_path_eval, metrics)

## Repo

In [ ]:
# cloning original repo
!git clone https://github.com/TharinduDR/DeepOffense

Cloning into 'DeepOffense'...
remote: Enumerating objects: 749, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 749 (delta 0), reused 0 (delta 0), pack-reused 746
Receiving objects: 100% (749/749), 5.58 MiB | 19.00 MiB/s, done.
Resolving deltas: 100% (494/494), done.


In [ ]:
%cd DeepOffense
!pwd
!pip install -r requirements.txt
#!unzip "/content/drive/My Drive/Paper_dataton/DeepOffense-English-XLM-R.zip" -d /content/DeepOffense/deepoffense/en/

/content/DeepOffense
/content/DeepOffense
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.4/125.4 KB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 37.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 72.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 100.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 KB 6.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.9/178.9 KB 24.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.0/184.0 KB 26.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 KB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 KB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 

### Test Fine tune

In [ ]:
!pip install GPUtil

import torch
from GPUtil import showUtilization as gpu_usage
from numba import cuda

def free_gpu_cache():
    print("Initial GPU Usage")
    gpu_usage()                             

    torch.cuda.empty_cache()

    cuda.select_device(0)
    cuda.close()
    cuda.select_device(0)

    print("GPU Usage after emptying the cache")
    gpu_usage()

free_gpu_cache()      

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for GPUtil: filename=GPUtil-1.4.0-py3-none-any.whl size=7409 sha256=a55391ea189521b0cb0fb17b3ece7be5160be74f63d8e376e01cb2c5a465421d
  Stored in directory: /root/.cache/pip/wheels/ba/03/bb/7a97840eb54479b328672e15a536e49dc60da200fb21564d53
Successfully built GPUtil
Initial GPU Usage
| ID | GPU | MEM |
------------------
|  0 |  0% |  0% |
GPU Usage after emptying the cache
| ID | GPU | MEM |
------------------
|  0 | 12% |  1% |


### Test Training

In [ ]:
import os
import shutil
import time
import csv
import numpy as np
import pandas as pd
import sklearn
import torch
from sklearn.model_selection import train_test_split

from deepoffense.classification import ClassificationModel
from deepoffense.language_modeling.language_modeling_model import LanguageModelingModel
from examples.common.evaluation import macro_f1, weighted_f1
from examples.common.label_converter import decode, encode
from examples.common.print_stat import print_information
from examples.turkish.turkish_deepoffense_config import LANGUAGE_FINETUNE, TEMP_DIRECTORY, SUBMISSION_FOLDER, \
    MODEL_TYPE, MODEL_NAME, language_modeling_args, args, SEED, RESULT_FILE

if not os.path.exists(TEMP_DIRECTORY):
  os.makedirs(TEMP_DIRECTORY)
if not os.path.exists(os.path.join(TEMP_DIRECTORY, SUBMISSION_FOLDER)):
  os.makedirs(os.path.join(TEMP_DIRECTORY, SUBMISSION_FOLDER))


def train(data_train, test_list, save_path):
  global LANGUAGE_FINETUNE, TEMP_DIRECTORY, SUBMISSION_FOLDER, MODEL_TYPE, MODEL_NAME, language_modeling_args, args, SEED, RESULT_FILE
  
  train = data_train
  train = train.rename(columns={'HS': 'labels'})
  train = train[['text', 'labels']]
  train['labels'].apply(int)
  print(MODEL_NAME)


  #Fine Tuning
  train, test = train_test_split(train, test_size=0.2)

  language_modeling_args['max_seq_length'] = 512
  args['max_seq_length'] = 512
  args["evaluate_during_training"] = False
  # This is the Fine tuning part
  if True:
      train_list_ = train['text'].tolist()
      test_list_ = test['text'].tolist()
      complete_list = train_list_ + test_list_
      lm_train = complete_list[0: int(len(complete_list)*0.8)]
      lm_test = complete_list[-int(len(complete_list)*0.2):]

      with open(os.path.join(TEMP_DIRECTORY, "lm_train.txt"), 'w') as f:
          for item in lm_train:
              f.write("%s\n" % item)

      with open(os.path.join(TEMP_DIRECTORY, "lm_test.txt"), 'w') as f:
          for item in lm_test:
              f.write("%s\n" % item)

      model = LanguageModelingModel('auto', MODEL_NAME, args=language_modeling_args)
      model.train_model(os.path.join(TEMP_DIRECTORY, "lm_train.txt"), eval_file=os.path.join(TEMP_DIRECTORY, "lm_test.txt"))
      MODEL_NAME = language_modeling_args["best_model_dir"]


  # Rebuilt train data and train without evaluation as they did in Multilingual Resources for Offensive Language Detection
  train = data_train
  train = train.rename(columns={'HS': 'labels'})
  train = train[['text', 'labels']]
  train['labels'].apply(int)

  # Train the model
  print("Started Training")

  train['labels'] = encode(train["labels"])

  print(MODEL_NAME)
  model = ClassificationModel(MODEL_TYPE,
                              MODEL_NAME,
                              args=args, 
                              use_cuda=torch.cuda.is_available())  # You can set class weights by using the optional weight argument
  model.train_model(train, macro_f1=macro_f1, weighted_f1=weighted_f1, accuracy=sklearn.metrics.accuracy_score)
  
  for test_ in test_list:
    test_df, lang_test = test_
    test_df = test_df.rename(columns={'HS': 'labels'})
    test_df = test_df if 'hatecheck' in lang_test else test_df[['text', 'labels']]
    test_df['labels'].apply(int)

    test_df['labels'] = encode(test_df["labels"])

    test_sentences = test_df['text'].tolist()

    predictions, raw_outputs = model.predict(test_sentences)
    test_df['predictions'] = predictions

    test_df['predictions'] = decode(test_df['predictions'])
    test_df['labels'] = decode(test_df['labels'])
    
    print(save_path + '_' + lang_test + '_results_II.csv')
    test_df.to_csv(save_path + '_' + lang_test + '_results_II.csv', header=True, index=False, encoding='utf-8')


def train_models(experiment):
  save_path_ = save_path + 'models/EMNLP20/'
  for lang, item in experiment.items():
    train(item['data_train'], item['data_test'], save_path_ + lang)



In [ ]:
#experiments_train = [
#    {'es': {'data_train': data_hateval_es_full,
#            'data_test': [(data_hateval_en_full, 'en'),
#                          (data_arango_train, 'cl1'),
#                          (data_full, 'cl2')],}},
#    {'en': {'data_train': data_hateval_en_full,
#            'data_test': [(data_hateval_es_full, 'es'),
#                          (data_arango_train, 'cl1'),
#                          (data_full, 'cl2')],}},
#    {'cl1': {'data_train': data_arango_train,
#             'data_test': [(data_hateval_es_full, 'es'),
#                          (data_hateval_en_full, 'en'),
#                          (data_full, 'cl2')],}},
#    {'cl2': {'data_train': data_full,
#             'data_test': [(data_hateval_es_full, 'es'),
#                          (data_hateval_en_full, 'en'),
#                          (data_arango_train, 'cl1')],}}
#    ]

experiments_train = [
    {'es': {'data_train': data_hateval_es_full,
            'data_test': [(data_hatecheck, 'es_hatecheck')],
            }},
    {'en': {'data_train': data_hateval_en_full,
            'data_test': [(data_hatecheck, 'es_hatecheck')],
            }},
    {'cl1': {'data_train': data_arango_train,
             'data_test': [(data_hatecheck, 'es_hatecheck')],
             }},
    {'cl2': {'data_train': data_full,
             'data_test': [(data_hatecheck, 'es_hatecheck')],
             }},
    {'clT': {'data_train': data_chilean,
             'data_test': [(data_hatecheck, 'es_hatecheck')],
             }}
    ]


train_models(experiments_train[-1])
torch.cuda.empty_cache()

xlm-roberta-large


  0%|          | 0/15353 [00:00<?, ?it/s]

  0%|          | 0/1054 [00:00<?, ?it/s]

Epoch:   0%|          | 0/2 [00:00<?, ?it/s]

Running Epoch 0 of 2:   0%|          | 0/132 [00:00<?, ?it/s]

...

Running Epoch 1 of 3:   0%|          | 0/1798 [00:00<?, ?it/s]

Running Epoch 2 of 3:   0%|          | 0/1798 [00:00<?, ?it/s]

  0%|          | 0/3745 [00:00<?, ?it/s]

  0%|          | 0/469 [00:00<?, ?it/s]

/content/drive/MyDrive/Paper_dataton/models_reproduction/models/EMNLP20/clT_es_hatecheck_results_II.csv


## Results

In [6]:
experiments_results = [
    {'es': ['en', 'cl1', 'cl2'],},
    {'en': ['es', 'cl1', 'cl2'],},
    {'cl1': ['es', 'en', 'cl2'],},
    {'cl2': ['es', 'en', 'cl1'],}
    ]

# Define Evaluation metrics
def compute_metrics(p):
    pred, labels = p

    accuracy = accuracy_score(y_true=labels, y_pred=pred)
    recall = recall_score(y_true=labels, y_pred=pred)
    precision = precision_score(y_true=labels, y_pred=pred)
    f1 = f1_score(y_true=labels, y_pred=pred)
    auc = roc_auc_score(y_true=labels, y_score=pred)

    return {"accuracy": accuracy, "precision": precision,
            "recall": recall, "f1": f1, "AUC": auc}

def load_results(experiments_results):
  results_dict = {}
  for experiment in experiments_results:
    for key_lang, item in  experiment.items():
      for lang in item:
        df = pd.read_csv(save_path + 'models/EMNLP20/' +  key_lang + "_" + lang + '_results_II.csv') if key_lang != 'es' else pd.read_csv(save_path + 'models/EMNLP20/' +  key_lang + "_" + lang + '_results.csv')
        l = df['labels'].to_numpy()
        p = df['predictions'].to_numpy()
        results_dict[ key_lang + "_" + lang] = compute_metrics((p, l))

  pd.DataFrame.from_dict(results_dict, orient='index').to_csv(save_path + 'EMNLP20_evaluation_results_finetuning.csv')
  display(pd.DataFrame.from_dict(results_dict, orient='index'))
  print(pd.DataFrame.from_dict(results_dict, orient='index').to_latex())

load_results(experiments_results)

In [8]:
from scipy.stats import pmean, hmean, gmean

bias_files = ['es_es_hatecheck_results_II.csv',
              'en_es_hatecheck_results_II.csv',
              'cl1_es_hatecheck_results_II.csv',
              'cl2_es_hatecheck_results_II.csv',
              'clT_es_hatecheck_results_II.csv']


# Define Evaluation metrics
def compute_metrics_II(p):
    pred, labels = p
    auc = roc_auc_score(y_true=labels, y_score=pred)

    return auc


def metrics(files):
  results_dict = dict()
  for file_name in files:
    df = pd.read_csv(save_path + 'models/EMNLP20/' + file_name, index_col=0)
    key_lang = file_name.split('_')[2]
    lang = file_name.split('_')[0]
    l = df['labels'].to_numpy()
    p = df['predictions'].to_numpy()
    results_dict[lang + '_' + key_lang] = compute_metrics((p, l))
  results_df = pd.DataFrame.from_dict(results_dict, orient='index')
  display(results_df)
  print(results_df.to_latex())



def bias_metrics(bias_files):
  results_dict = dict()
  for file_ in bias_files:
    df = pd.read_csv(save_path + 'models/EMNLP20/' + file_, index_col=0)
    key_lang = file_.split('_')[2]
    lang = file_.split('_')[0]
    #print(df.columns.tolist())
    #print(df['target_ident'].unique()[:-1])
    for col in df['target_ident'].unique()[:-1]:
      dicc = dict()
      # subgroup AUC
      df_ = df[df['target_ident'] == col]
      l = df_['labels'].to_numpy()
      p = df_['predictions'].to_numpy()
      dicc["Subgroup AUC"] = compute_metrics_II((p, l))
      # BPSN AUC
      df_ = df[(df['target_ident'] == col) & (df['labels'] == 0) | (df['target_ident'] != col) & (df['labels'] == 1)]
      l = df_['labels'].to_numpy()
      p = df_['predictions'].to_numpy()
      dicc['BPSN AUC'] = compute_metrics_II((p, l))
      # BNSP AUC
      df_ = df[(df['target_ident'] == col) & (df['labels'] == 1) | (df['target_ident'] != col) & (df['labels'] == 0)]
      l = df_['labels'].to_numpy()
      p = df_['predictions'].to_numpy()
      dicc['BNSP AUC'] = compute_metrics_II((p, l))
      results_dict[ key_lang + "_" + lang + '_' + col] = dicc
  
  display(pd.DataFrame.from_dict(results_dict, orient='index'))
  print(pd.DataFrame.from_dict(results_dict, orient='index').to_latex())
  dict_results = dict()
  # GMB AUC
  for like in ['es_', 'en_', 'cl1_', 'cl2_', 'clT_']:
    df = pd.DataFrame.from_dict(results_dict, orient='index').filter(like=like, axis=0)
    dict_temp = dict()
    for col in df.columns.tolist():
      dict_temp['GMB ' + col] = pmean(df[col].tolist(), -5)
    dict_results[like.strip('_')] = dict_temp
  display(pd.DataFrame.from_dict(dict_results, orient='index'))
  print(pd.DataFrame.from_dict(dict_results, orient='index').to_latex())


bias_metrics(bias_files)
metrics(bias_files)

,Subgroup AUC,BPSN AUC,BNSP AUC
hatecheck_es_black people,0.511812,0.324024,0.885545
hatecheck_es_gay people,0.659711,0.834513,0.523291
hatecheck_es_trans people,0.688025,0.815068,0.565414
hatecheck_es_indigenous people,0.727885,0.711653,0.697332
hatecheck_es_jews,0.670595,0.576391,0.779166
hatecheck_es_disabled people,0.671840,0.760079,0.604463
hatecheck_es_women,0.644709,0.514918,0.816683
hatecheck_en_black people,0.737503,0.637355,0.730870
hatecheck_en_gay people,0.604747,0.693769,0.564004
hatecheck_en_trans people,0.618124,0.681775,0.586586


\begin{tabular}{lrrr}
\toprule
{} &  Subgroup AUC &  BPSN AUC &  BNSP AUC \\
\midrule
hatecheck\_es\_black people       &      0.511812 &  0.324024 &  0.885545 \\
hatecheck\_es\_gay people         &      0.659711 &  0.834513 &  0.523291 \\
hatecheck\_es\_trans people       &      0.688025 &  0.815068 &  0.565414 \\
hatecheck\_es\_indigenous people  &      0.727885 &  0.711653 &  0.697332 \\
hatecheck\_es\_jews               &      0.670595 &  0.576391 &  0.779166 \\
hatecheck\_es\_disabled people    &      0.671840 &  0.760079 &  0.604463 \\
hatecheck\_es\_women              &      0.644709 &  0.514918 &  0.816683 \\
hatecheck\_en\_black people       &      0.737503 &  0.637355 &  0.730870 \\
hatecheck\_en\_gay people         &      0.604747 &  0.693769 &  0.564004 \\
hatecheck\_en\_trans people       &      0.618124 &  0.681775 &  0.586586 \\
hatecheck\_en\_indigenous people  &      0.599963 &  0.699432 &  0.554616 \\
hatecheck\_en\_jews               &      0.660294 &  0.670747 &  0.

,GMB Subgroup AUC,GMB BPSN AUC,GMB BNSP AUC
es,0.628262,0.460510,0.632194
en,0.633511,0.605110,0.600825
cl1,0.500210,0.500190,0.500210
cl2,0.609366,0.531649,0.652904
clT,0.566642,0.430491,0.484545


\begin{tabular}{lrrr}
\toprule
{} &  GMB Subgroup AUC &  GMB BPSN AUC &  GMB BNSP AUC \\
\midrule
es  &          0.628262 &      0.460510 &      0.632194 \\
en  &          0.633511 &      0.605110 &      0.600825 \\
cl1 &          0.500210 &      0.500190 &      0.500210 \\
cl2 &          0.609366 &      0.531649 &      0.652904 \\
clT &          0.566642 &      0.430491 &      0.484545 \\
\bottomrule
\end{tabular}



,accuracy,precision,recall,f1,AUC
es_hatecheck,0.675033,0.845628,0.657675,0.739902,0.686879
en_hatecheck,0.535915,0.909341,0.377280,0.533298,0.644165
cl1_hatecheck,0.297463,1.000000,0.000380,0.000760,0.500190
cl2_hatecheck,0.710814,0.830277,0.739742,0.782399,0.691075
clT_hatecheck,0.550067,0.822768,0.458587,0.588924,0.612492


\begin{tabular}{lrrrrr}
\toprule
{} &  accuracy &  precision &    recall &        f1 &       AUC \\
\midrule
es\_hatecheck  &  0.675033 &   0.845628 &  0.657675 &  0.739902 &  0.686879 \\
en\_hatecheck  &  0.535915 &   0.909341 &  0.377280 &  0.533298 &  0.644165 \\
cl1\_hatecheck &  0.297463 &   1.000000 &  0.000380 &  0.000760 &  0.500190 \\
cl2\_hatecheck &  0.710814 &   0.830277 &  0.739742 &  0.782399 &  0.691075 \\
clT\_hatecheck &  0.550067 &   0.822768 &  0.458587 &  0.588924 &  0.612492 \\
\bottomrule
\end{tabular}



In [ ]:
files = ['es_es_hatecheck_results_II.csv',
         'en_es_hatecheck_results_II.csv',
         'cl1_es_hatecheck_results_II.csv',
         'cl2_es_hatecheck_results_II.csv',
         'clT_es_hatecheck_results_II.csv']

def hatecheck(files):
  dicc_ = dict()
  dicc__2 = dict()
  for file_ in files:
    dicc = dict()
    df = pd.read_csv(save_path + "models/EMNLP20/" + file_)
    df['predictions'] = df['predictions'].apply(int)
    filters = df['functionality'].unique()
    for filter in filters:
      df_ = df[df['functionality'] == filter]
      dicc[filter] = accuracy_score(df_['labels'].values, df_['predictions'].values)
    dicc_2 = dict()
    filters = df['target_ident'].unique()
    for filter in filters:
      df_ = df[df['target_ident'] == filter]
      dicc_2[filter] = f1_score(df_['labels'].values, df_['predictions'].values)
    key = file_.split('_')[0]
    print(key, file_)
    dicc_[key] = dicc
    dicc__2[key] = dicc_2
  df = pd.DataFrame.from_dict(dicc_, orient='index').transpose()
  print(df.columns.tolist())
  df['delta_cl'] = df['cl2'] - df['cl1']
  display(df)
  print(df.to_latex())
  df = pd.DataFrame.from_dict(dicc__2, orient='index').transpose()
  print(df.columns.tolist())
  df['delta_cl'] = df['cl2'] - df['cl1']
  # dropping the rows having NaN values
  df = df.drop(index=df.index.tolist()[-1])
  display(df)
  print(df.to_latex())

hatecheck(files)

es es_es_hatecheck_results_II.csv
en en_es_hatecheck_results_II.csv
cl1 cl1_es_hatecheck_results_II.csv
cl2 cl2_es_hatecheck_results_II.csv
clT clT_es_hatecheck_results_II.csv
['es', 'en', 'cl1', 'cl2', 'clT']


,es,en,cl1,cl2,clT,delta_cl
derog_neg_emote_h,0.585714,0.235714,0.000000,0.685714,0.478571,0.685714
derog_neg_attrib_h,0.757143,0.335714,0.007143,0.892857,0.600000,0.885714
derog_dehum_h,0.807143,0.492857,0.000000,0.828571,0.671429,0.828571
derog_impl_h,0.614286,0.314286,0.000000,0.621429,0.407143,0.621429
threat_dir_h,0.757143,0.557143,0.000000,0.707143,0.450000,0.707143
threat_norm_h,0.793103,0.868966,0.000000,0.924138,0.517241,0.924138
slur_h,0.533333,0.306667,0.000000,0.526667,0.360000,0.526667
profanity_h,0.728571,0.428571,0.000000,0.892857,0.564286,0.892857
profanity_nh,0.950000,0.930000,1.000000,1.000000,1.000000,0.000000
ref_subs_clause_h,0.634483,0.344828,0.000000,0.813793,0.531034,0.813793


\begin{tabular}{lrrrrrr}
\toprule
{} &        es &        en &       cl1 &       cl2 &       clT &  delta\_cl \\
\midrule
derog\_neg\_emote\_h  &  0.585714 &  0.235714 &  0.000000 &  0.685714 &  0.478571 &  0.685714 \\
derog\_neg\_attrib\_h &  0.757143 &  0.335714 &  0.007143 &  0.892857 &  0.600000 &  0.885714 \\
derog\_dehum\_h      &  0.807143 &  0.492857 &  0.000000 &  0.828571 &  0.671429 &  0.828571 \\
derog\_impl\_h       &  0.614286 &  0.314286 &  0.000000 &  0.621429 &  0.407143 &  0.621429 \\
threat\_dir\_h       &  0.757143 &  0.557143 &  0.000000 &  0.707143 &  0.450000 &  0.707143 \\
threat\_norm\_h      &  0.793103 &  0.868966 &  0.000000 &  0.924138 &  0.517241 &  0.924138 \\
slur\_h             &  0.533333 &  0.306667 &  0.000000 &  0.526667 &  0.360000 &  0.526667 \\
profanity\_h        &  0.728571 &  0.428571 &  0.000000 &  0.892857 &  0.564286 &  0.892857 \\
profanity\_nh       &  0.950000 &  0.930000 &  1.000000 &  1.000000 &  1.000000 &  0.000000 \\
ref\_subs\_clau

,es,en,cl1,cl2,clT,delta_cl
black people,0.863459,0.700565,0.005917,0.711986,0.573770,0.706069
gay people,0.524954,0.367010,0.000000,0.821596,0.804938,0.821596
trans people,0.604167,0.420635,0.000000,0.790257,0.788991,0.790257
indigenous people,0.780347,0.343816,0.000000,0.840964,0.816121,0.840964
jews,0.834337,0.523810,0.000000,0.881633,0.275510,0.881633
disabled people,0.651466,0.292473,0.000000,0.620102,0.044665,0.620102
women,0.850633,0.890339,0.000000,0.760606,0.240363,0.760606


\begin{tabular}{lrrrrrr}
\toprule
{} &        es &        en &       cl1 &       cl2 &       clT &  delta\_cl \\
\midrule
black people      &  0.863459 &  0.700565 &  0.005917 &  0.711986 &  0.573770 &  0.706069 \\
gay people        &  0.524954 &  0.367010 &  0.000000 &  0.821596 &  0.804938 &  0.821596 \\
trans people      &  0.604167 &  0.420635 &  0.000000 &  0.790257 &  0.788991 &  0.790257 \\
indigenous people &  0.780347 &  0.343816 &  0.000000 &  0.840964 &  0.816121 &  0.840964 \\
jews              &  0.834337 &  0.523810 &  0.000000 &  0.881633 &  0.275510 &  0.881633 \\
disabled people   &  0.651466 &  0.292473 &  0.000000 &  0.620102 &  0.044665 &  0.620102 \\
women             &  0.850633 &  0.890339 &  0.000000 &  0.760606 &  0.240363 &  0.760606 \\
\bottomrule
\end{tabular}

